In [174]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

import numpy as np
import pandas as pd
from keras.layers import Embedding, Reshape, merge, Dropout, Dense, Input, Flatten
from keras.models import Model
from keras.optimizers import Adamax

from collections import Counter


import matplotlib.pyplot as plt
%matplotlib inline

In [176]:
ratings = pd.read_csv('data/ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [177]:
# topmovies = Counter(ratings['movieId'])
# topmovies_idx =  [i for i,k in topmovies.most_common(50)]
# movies[movies['movieId'].isin(topmovies_idx)]

In [178]:
# ratings = ratings[ratings['movieId'].isin(topmovies_idx)].copy()
# df = movies[movies['movieId'].isin(topmovies_idx)].copy()
# ratings = ratings.sample(100000)

In [237]:
movie_names = pd.read_csv('data/movies.csv').set_index('movieId')['title'].to_dict()

In [179]:
uid = list(ratings['userId'].unique())
mid = list(ratings['movieId'].unique())

In [190]:
userid2idx = {o:i for i,o in enumerate(uid)}
movieid2idx = {o:i for i,o in enumerate(mid)}

# We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

new_movie_names = {ni:movie_names[oi] for (oi,ni) in movieid2idx.items()}

In [191]:
userinp = ratings['userId'].values
movieinp = ratings['movieId'].values
target = ratings['rating'].values

In [193]:
print(len(uid),len(mid))
ratings.describe()
# uid[-5:],mid[-5:]


671 9066


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,346.011310,1660.778349,3.543608,1.129639e+09
std,195.163838,1893.955817,1.058064,1.916858e+08
min,0.000000,0.000000,0.500000,7.896520e+08
25%,181.000000,327.000000,3.000000,9.658478e+08
50%,366.000000,873.000000,4.000000,1.110422e+09
75%,519.000000,2344.000000,4.000000,1.296192e+09
max,670.000000,9065.000000,5.000000,1.476641e+09


In [209]:
from keras.regularizers import l2

userid = Input(shape=(1,),name='user')
movieid = Input(shape=(1,),name='movie')
ub = Embedding(len(uid),1,input_length=1)(userid)
ub = Flatten()(ub)
uw = Embedding(len(uid),100,input_length=1,W_regularizer=l2(1e-4))(userid)
# x1 = Reshape((100,))(x1)
mb = Embedding(len(uid),1,input_length=1)(movieid)
mb = Flatten()(mb)
mw = Embedding(len(mid),100,input_length=1,W_regularizer=l2(1e-4))(movieid)
# x2= Reshape((100,))(x2)
x = merge([uw,mw],mode='dot')
x = Flatten()(x)
x = merge([x,ub],mode='sum')
x = merge([x,mb],mode='sum')
# x = Dropout(0.2)(x)
# x = Dense(100,activation='relu')(x)
# x = Dropout(0.2)(x)
# x = Dense(1,activation='linear')(x)
model = Model([userid,movieid],x)
model.summary()

model.compile(optimizer='adamax', loss='mse')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user (InputLayer)                (None, 1)             0                                            
____________________________________________________________________________________________________
movie (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_53 (Embedding)         (None, 1, 100)        67100       user[0][0]                       
____________________________________________________________________________________________________
embedding_55 (Embedding)         (None, 1, 100)        906600      movie[0][0]                      
___________________________________________________________________________________________

/home/chenjennhaur/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(671, 100, input_length=1, embeddings_regularizer=<keras.reg...)`
/home/chenjennhaur/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(9066, 100, input_length=1, embeddings_regularizer=<keras.reg...)`
/home/chenjennhaur/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/chenjennhaur/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=

In [210]:
model.fit([userinp,movieinp], target,epochs=5, batch_size=128)

Epoch 1/5
100004/100004 [==============================] - 1s - loss: 12.6314     
Epoch 2/5
100004/100004 [==============================] - 1s - loss: 8.8627     
Epoch 3/5
100004/100004 [==============================] - 1s - loss: 4.9920     
Epoch 4/5
100004/100004 [==============================] - 1s - loss: 3.4213     
Epoch 5/5
100004/100004 [==============================] - 1s - loss: 2.6913     


In [215]:
for l,m in enumerate(model.layers):
    print(l,m)

0 <keras.engine.topology.InputLayer object at 0x7fdfd2f76278>
1 <keras.engine.topology.InputLayer object at 0x7fdfd2f76208>
2 <keras.layers.embeddings.Embedding object at 0x7fdfd2f86e80>
3 <keras.layers.embeddings.Embedding object at 0x7fdfd2fdb9b0>
4 <keras.legacy.layers.Merge object at 0x7fdfd2f90828>
5 <keras.layers.embeddings.Embedding object at 0x7fdfd2f766a0>
6 <keras.layers.core.Flatten object at 0x7fdfd2fa42e8>
7 <keras.layers.core.Flatten object at 0x7fdfd2f7dd30>
8 <keras.layers.embeddings.Embedding object at 0x7fdfd2f86f98>
9 <keras.legacy.layers.Merge object at 0x7fdfd2f39b38>
10 <keras.layers.core.Flatten object at 0x7fdfd2f68e10>
11 <keras.legacy.layers.Merge object at 0x7fdfd2f37eb8>


In [212]:
# from keras import backend as K
# get_movie_embed = K.function([model.layers[0].input],[model.layers[3].output])
# layer_output = get_movie_embed([userinp,movieinp])[0]

In [229]:
movie_latent = model.layers[3].get_weights()
movie_latent[0][1]
# Sama
# get_movie_emb = Model(movieid,mw)
# movie_emb = np.squeeze(get_movie_emb.predict(movieinp))
# movie_emb = np.squeeze(get_movie_emb.predict(topMovies))

array([ 0.10168266,  0.1279833 , -0.08375041, -0.1086758 , -0.11080393,
        0.11922451, -0.12217389,  0.10011319,  0.09329143, -0.11516085,
       -0.1028003 , -0.13553102,  0.10699932,  0.09149715,  0.14782044,
        0.1097913 , -0.04413486,  0.1729245 , -0.11060912,  0.08953319,
       -0.11967392, -0.12070232, -0.17658986, -0.10651191, -0.13624547,
       -0.11130563,  0.10838658, -0.13793446, -0.09037676,  0.11579714,
        0.07717948, -0.08035419, -0.10647401,  0.11677355, -0.11622167,
        0.10576204, -0.11023118, -0.11258791, -0.09275565,  0.13806061,
        0.14974687, -0.10317611,  0.13980848,  0.14472876, -0.14166625,
       -0.08852642, -0.14499663,  0.10514073, -0.1299447 , -0.08775359,
       -0.0903061 , -0.15199898,  0.07470693,  0.09717508, -0.11475866,
        0.14287427,  0.12215807, -0.12951049,  0.10131739,  0.08181577,
       -0.10441489, -0.09535695, -0.07205888, -0.11140583, -0.07833101,
       -0.10066409, -0.12113469,  0.09158402, -0.08592007, -0.12

In [219]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:2000]
topMovies = np.array(topMovies.index)

In [221]:
topMovies[:10]

array([ 57,  49,  99,  92, 143,  72, 402, 417,  79,  89])

In [233]:
ratings.groupby('movieId')['rating'].count().sort_values(ascending=False)[:10]

movieId
57     341
49     324
99     311
92     304
143    291
72     274
402    259
417    247
79     244
89     237
Name: rating, dtype: int64

In [251]:
from sklearn.manifold import TSNE

movie_tsne = TSNE(n_components=2).fit_transform(movie_latent[0])


In [261]:
df = pd.DataFrame.from_dict(new_movie_names,orient='index')
df['x'] = movie_tsne[:,0]
df['y'] = movie_tsne[:,1]
df.head(2)

,0,x,y
0,Dangerous Minds (1995),4.529202,18.516392
1,Dumbo (1941),-8.803856,17.259270


In [290]:
# ax1.set_xlim([0, 5])
plt.figure(figsize=(128, 128)) 
xcoord = df['x']
ycoord = df['y']
labels = df[0].values
# marker = {0:"o",1:"1",2:"*"}
# color = {0:"black",1:"red",2:"yellow"}

for i in range(len(xcoord)):
#     plt.scatter(xcoord[i],ycoord[i],color=color[word_type[i]],marker=marker[word_type[i]])
    plt.scatter(xcoord[i],ycoord[i])
    plt.annotate(labels[i],xy=(xcoord[i], ycoord[i]),xytext=(5, 2),textcoords='offset points',ha='right',va='bottom')


plt.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [276]:
get_movie_bias = Model(movieid, mb)
movie_bias = get_movie_bias.predict([i for i in range(0,len(mid))])

In [278]:
movie_bias

array([[ 0.21145828],
       [ 0.15131749],
       [ 0.13508104],
       ..., 
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]], dtype=float32)

In [279]:
df['bias'] = movie_bias

In [289]:
print(df.sort_values(by='bias',ascending=True)[:10])
print(df.sort_values(by='bias',ascending=False)[:10])

                                     0          x          y      bias
572               Life in a Day (2011)  40.226490   1.341512 -0.036607
617             Art of War, The (2000)  35.798302  13.974094 -0.034724
606             Mad About Mambo (2000)  35.958530  -5.401708 -0.030838
557            Step Into Liquid (2002) -33.609989  28.579762 -0.021811
596              Rocketship X-M (1950)  35.456257  -7.150631 -0.019564
296                  Psycho III (1986)  27.673645  27.143049 -0.019214
619           Steal This Movie! (2000) -61.570492  30.103573 -0.011952
501                    Firewall (2006) -43.270344   2.698572 -0.008327
500                   Annapolis (2006)  35.043846  17.466772 -0.007550
248  Return from Witch Mountain (1978) -47.095795 -10.839712 -0.006922
                                             0          x          y      bias
49                         Pulp Fiction (1994)   9.589172  26.989519  0.608185
57                         Forrest Gump (1994)  10.105489  26

In [ ]:
# cat_maps = [(o, LabelEncoder()) for o in cat_vars]
# contin_maps = [([o], StandardScaler()) for o in contin_vars]

# cat_mapper = DataFrameMapper(cat_maps)
# cat_map_fit = cat_mapper.fit(joined)
# cat_cols = len(cat_map_fit.features)